# Installation

In [ ]:
# 1) Install ultralytics if needed (comment out if already installed)
# %pip install ultralytics
# %pip install pandas
# %pip install tabulate   # optional, if you want tabulate

In [ ]:
# Update the settings to point to the datasets and runs directories
from ultralytics import settings
from ct_detector import DATASETS_DIR, ROOT_DIR
import os

settings.update({'datasets_dir': DATASETS_DIR, 'runs_dir': os.path.join(ROOT_DIR, 'runs')})

# Evaluate single model

In [ ]:
# 1) evaluate a single model
import pandas as pd
from tabulate import tabulate
from ct_detector.model import evaluate_model
from ct_detector.model import MODELS, DATASETS

# 2) (Optional) If you have a GPU and want to specify it, set device='0' or 'cpu'
model_path = MODELS['eie_t_1_yolov8m']  # or your custom model: 'path/to/best.pt'
data_path  = DATASETS.get("1", None)  # or your dataset .yaml or folder of images
device = '' # '' or 'cpu for CPU, '0' for GPU 0, '1' for GPU 1, etc.

# 4) Run evaluation
results = evaluate_model(
    model_path=model_path,
    data_path=data_path,
    conf=0.25,
    iou=0.45,
    device=device,
    imgsz=640,
    save_json=False,
    project='runs/model_evaluation',
    name='demo_evaluate',
    verbose=True # True to print results, False to suppress
)
# 5) (Optional) If you want detailed outputs, you can look at results.keys():
print("\nAll results keys:", dir(results))

In [ ]:
# 6) (Optional) If you want to display confusion matrix you can do this:
from ct_detector.display import display_confusion_matrix
from functools import partial

callback = partial(display_confusion_matrix, width=640, height=640)
results.confusion_matrix.plot(normalize=False, names=tuple(c for c in results.names.values()), on_plot=callback)

# Evaluate multiple models

In [ ]:
# 2) compare models
from ct_detector.model import compare_models
from ct_detector.model import MODELS, DATASETS

# Print all available models
print("Available models:")
for model_name, model_path in MODELS.items():
    print(f"{model_name}: {model_path}")

# Print all available datasets
print("\nAvailable datasets:")
for dataset_name, dataset_path in DATASETS.items():
    print(f"{dataset_name}: {dataset_path}")

# Create a list of model paths to compare from the MODELS dictionary that automatically gets all available models
model_list = [path for path in MODELS.values() if path.endswith('.pt')]
#model_list = [MODELS['eie_t_1_yolov8m']]  # Add more models as needed

# Alternatively, you can specify specific models like this:
# model_list = [MODELS['yolov8n.pt'], MODELS['yolov8s.pt']]  # Add more models as needed

# Get the default dataset path from the DATASETS dictionary which is a dictionary of available datasets
data_path = DATASETS.get("1", None)  # Or 'path/to/yourData.yaml'

results = compare_models(
    model_paths=model_list,
    data_path=data_path,
    conf=0.25,
    iou=0.45,
    device='',   # GPU if available, else '' or 'cpu'
    imgsz=640,
    project='runs/model_comparison',
    name='demo_compare',
    verbose=False
)

# results is a dictionary of results for each model where keys are model names and values are the results
print("\n--- Retrieved Evaluation Results For: ---")
for key in results.keys():
    print(f"\n--- {key} ---")